# Model Training Notebook

In [2]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
from IPython.core.display import display, HTML
import cv2
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import ast

from sklearn.model_selection import train_test_split

# repeatable functions housed in the utils file and imported here
from utils import *
from model_training_utils import train as training_loop
from model_training_utils import validate as validation_loop
from model_training_utils import hp_grid_search
from models import CNN
from models import YOLO

## Create Training and Validation Datasets

In [3]:
df= pd.read_csv('./datasets/annotations_map.csv', converters={'new_bb': from_np_array})

In [4]:
df_train = df.reset_index()
X = df_train[['new_path','new_bb']]
Y = df_train['class']
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.25, random_state=42)
print('Number of training observations: ', X_train.shape[0])
print('Number of validation observations: ', X_val.shape[0])

Number of training observations:  32
Number of validation observations:  11


## Build RCNN

## Build YOLO

#### The Design of the YOLO NN was taken from the following paper:

https://arxiv.org/pdf/1506.02640.pdf - "You Only Look Once: Unified, Real-Time Object Detection" by Redmon, Divvala, Girshick, and Farhadi

The following article is YOLO V2:
https://arxiv.org/pdf/1612.08242v1.pdf - "YOLO 9000: Better, Faster, Stronger" by Redmon, and Farhadi

In [17]:
class YOLO(nn.Module):
    
    def __init__(self):
        super(YOLO, self).__init__()
        
        # First
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding=1)
        self.pooling = nn.AvgPool2d(2, 2)
        
        # Second
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding=1)
        
        # Third
        self.conv3 = nn.Conv2d(64, 128, 3, 1, padding=1)
        self.conv4 = nn.Conv2d(128, 64, 1, 1)
        
        # Fourth
        self.conv5 = nn.Conv2d(128, 256, 3, 1, padding=1)
        self.conv6 = nn.Conv2d(256, 128, 1, 1)
        
        # Fifth
        self.conv7 = nn.Conv2d(256, 512, 3, 1, padding=1)
        self.conv8 = nn.Conv2d(512, 256, 1, 1)
        
        # Sixth
        self.conv9 = nn.Conv2d(512, 1024, 3, 1, padding=1)
        self.conv10 = nn.Conv2d(1024, 512, 1, 1)
        
        # Final
        self.conv11 = nn.Conv2d(1024, 1000, 1, 1)
        
        # FC Layer and Softmax
        self.FC = nn.Linear(1000, 4)
        
        
            
    def forward(self, x):
        
        batch_size, h, w, channel = x.shape
        x= x.reshape(batch_size, channel, h, w)
        
        output = self.conv1(x) # 500
        output = self.pooling(output) # 250
        
        output = self.conv2(output) # 250
        output = self.pooling(output) # 125
        
        output = self.conv3(output) # 125
        output = self.conv4(output) # 125
        output = self.conv3(output) # 125
        output = self.pooling(output) # 62
        
        output = self.conv5(output) # 62
        output = self.conv6(output) # 62
        output = self.conv5(output) # 62
        output = self.pooling(output) # 31
        
        output = self.conv7(output) # 31
        output = self.conv8(output) # 31
        output = self.conv7(output) # 31
        output = self.conv8(output) # 31
        output = self.conv7(output) # 31
        output = self.pooling(output) # 15
        
        output = self.conv9(output) # 15
        output = self.conv10(output) # 15
        output = self.conv9(output) # 15
        output = self.conv10(output) # 15
        output = self.conv9(output) # 15
        
        output = self.conv11(output)
        output = output.mean([2, 3])
        
        output = self.FC(output)
        
        return output

# Train the Model


- Structured similarly to main.py file from pytorch part of A2

In [18]:
## Define Hyperparameters -- Currently setting values that we can modify
loss_type = "l1"
learning_rate = 0.001
momentum = 0.5
reg = 1e-2

training_batch_size= 5
validation_batch_size= 5

model= "YOLO"

In [19]:
if loss_type == "l1":
    criterion = nn.L1Loss()
    
if loss_type == "l2":
    criterion = nn.MSELoss()
    
if model== "SimpleCNN":
    model= CNN()
elif model == "YOLO":
    model = YOLO()
    
train_ds = WaldoDataset(X_train['new_path'],X_train['new_bb'] ,y_train)
valid_ds = WaldoDataset(X_val['new_path'],X_val['new_bb'],y_val)

train_dl = DataLoader(train_ds, batch_size=training_batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=validation_batch_size)

optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum,
                            weight_decay=reg)

In [20]:
training_loop(model= model, optimizer = optimizer, train_dl= train_dl, valid_dl=valid_dl, epochs= 20, criterion= criterion, verbose= True, return_loss= False)

Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([2, 4])
 
--------------------------------------------------------
Training Loss for Epoch 0: 1505.1929473876953
Output:  torch.Size([5, 4])


/Users/anandsongvit/CS7643/Final Project/CS7643-Final-Project/model_training_utils.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb= torch.tensor(y_bb)


Output:  torch.Size([5, 4])
Output:  torch.Size([1, 4])
Validation Loss for Epoch 0: 617.2049713134766
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([2, 4])
 
--------------------------------------------------------
Training Loss for Epoch 1: 1532.519790649414
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([1, 4])
Validation Loss for Epoch 1: 617.1839294433594
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([2, 4])
 
--------------------------------------------------------
Training Loss for Epoch 2: 1525.0445861816406
Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([1, 4])
Validation Loss for Epoch 2: 617.1627807617188
Output:  torch.Size([5, 4])
Output

Output:  torch.Size([5, 4])
Output:  torch.Size([5, 4])
Output:  torch.Size([1, 4])
Validation Loss for Epoch 19: 616.79248046875


In [ ]:
loss_type= ["l1"]
learning_rate= [0.001,0.0001]
momentum = [0.9]
reg = [0.01]
batch_size= [10]

all_training_loss, all_validation_loss= hp_grid_search(model_type= "SimpleCNN", 
               lr_list=learning_rate, 
               momentum_list=momentum, 
               reg_list=reg, 
               batch_size_list=batch_size,
               train_ds= train_ds,
               valid_ds= valid_ds,
               optimizer= optimizer, 
               loss_type_list=loss_type,
               epochs= 10,
               save_all_plots="Yes", 
               save_final_plot="Yes",
               final_plot_prefix="Test", 
               return_all_loss= True)